Neural Network will be 3 layers dense. 16x8x4   16 inputs, each corresponding to a square in the 4x4 grid. 4 outputs,
each corresponding to a one action ( up down left right)

Activation = Hyperbolic Tangent 

Fitness = Score = 10*Max + 1*sum(all_numbers_on_board)

Genome = Each weight and bias of the NN is mapped to a gene. Randomly changed

Tournament selection w/o replacement. Higher fitness genomes have more chance to be selected. Best should survive 100%.

Crossover = Crossover is done on selected parents to produce offspring

Mutation = Randomly mutate each child. 

Evaluate fitness and Start again. 

Each genome is 1 NN architecture with random weights and biases. Each genome is then allowed to play the game. After N
seconds or moves, fitness is evaluated. Each generation can be 10 - 100 Random Genomes. 


survival_rate = 0.5 
mutation_rate = 0.001
crossover_rate = 0.7
n_agents = 1500

In [2]:
import numpy as numpy
import matplotlib.pyplot as plt
import numpy as np
from Evolver2048 import Genome, Mutator
from SelectionProcess import rouletteSelection, skewnessSelection

In [4]:
def spawn_population(p):
	population = []
	for i in range(0,p):
		individual= Genome.fromRandom()
		population.append(individual)
	return(np.array(population))

def createChildren(population, numberOfChildren, selection, mutationRate=0.2, mutationSize=1, crossoverRate=0.5):
    children = []
    for j in range(numberOfChildren):
        # Select 2 parents from the list
        mother, father = selection(population, 2)
        # Crossover to create child
        child = Mutator.crossover(mother, father, rate=crossoverRate)
        # Mutate the genes of the child
        child = Mutator.mutate(child, rate=mutationRate, size=mutationSize)
        # Add to population
        children.append(child)
    return children

def savePopulation(population, path="lastPopulation"):
	genesList = [genome.genes for genome in population]
	genes = np.stack(genesList)
	np.save(path, genes)

def loadPopulation(path="./lastPopulation.npy"):
	genes = np.load(path)
	return [Genome.fromArray(geneArray) for geneArray in genes]

In [ ]:
"""
This is the population evolution loop.
To run successfully, you require this notebook to have a folder named genomes in the same directory, else genomes can't save successfully.
"""
# The evolver will stop after this many generations
generations = 300
# The size of the population each generation
populationSize = 100
# How many new genomes are made each generation
numberOfChildren = 20
# As each child replaces one genome
numberOfSurvivors = populationSize - numberOfChildren
# How many generations between each saved genome
saveFrequency = 10
# The selection function, including its bias
selection = lambda pop, n: rouletteSelection(pop, n, 1.5)
# The probability for each gene to be mutated
mutationRate = 0.5
# The size of the mutations
mutationSize = 2
# The chance for each gene to be inherited from the father. Values close to 0 or 1 produce children similar to one parent.
crossoverRate = 0.5
# Force the fitness function to simulate multiple games returning the average  
Genome.fitnessTrials = 10

# Time taken per generation is loosely proportional to numberOfChildren * fitnessTrials with generation 0 taking much longer

# Create initial random population
print(f"Initialising population of {populationSize}\n")
population = spawn_population(populationSize)

# Create lists to store fitnesses
bestFitnesses = []
meanFitnesses = []

for i in range(generations):
    print(f"Beginning generation {i}")

    # Eliminate weak candidates by keeping strong ones
    print(f"- Eliminating weakest {numberOfChildren}")
    survivors = selection(population, numberOfSurvivors)

    # Create population of children
    print(f"- Creating {numberOfChildren} children")
    children = createChildren(population, numberOfChildren, selection, mutationRate, mutationSize, crossoverRate)

    # Create population from survivors and children
    population = survivors + children # Python lists use + to concatenate

    # Report best fitness achieved
    best = max(population, key=lambda genome: genome.fitness())
    bestFit = best.fitness()
    bestFitnesses.append(bestFit)
    # Report mean fitness
    meanFit = sum([genome.fitness() for genome in population]) / populationSize
    meanFitnesses.append(meanFit)
    # If mean fitness is good save population
    if meanFit == max(meanFitnesses):
        savePopulation(population)

    # Save every so often
    if i % saveFrequency == 0:
        best.save(f"./genomes/generation{i}fitness{bestFit}.npy")
    # Save on last generation
    if i == generations - 1:
        best.save(f"./genomes/finalfitness{bestFit}.npy")

    print(f"End of generation {i}, best fitness achieved is {bestFit}, mean is {meanFit}\n")

In [ ]:
plt.plot(meanFitnesses)